In [32]:
import os
print(os.getcwd())
os.chdir('d:\\vscode_machineLearning\\internship\\Customer-Churn-Prediction')
print(os.getcwd())

d:\vscode_machineLearning\internship\Customer-Churn-Prediction
d:\vscode_machineLearning\internship\Customer-Churn-Prediction


In [33]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

In [34]:
df = pd.read_csv(f'artifacts/generated_data/442_data.csv')

In [35]:
df.isnull().sum()

Houston                       0
LosAngeles                    0
Miami                         0
NewYork                       0
Age                           0
Gender                        0
Subscription_Length_Months    0
Monthly_Bill                  0
Total_Usage_GB                0
Churn                         0
dtype: int64

In [36]:
encoder = LabelEncoder()

In [37]:
df['Gender']=df['Gender'].replace({'Male':0,'Female':1})
# df['Location'] = encoder.fit_transform(df[['Location']])

In [38]:
X = df.drop(columns='Churn')
y = df['Churn']

In [39]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (OneHotEncoder,
                                   MinMaxScaler)
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

In [40]:
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.20, random_state=42)

In [41]:
X_train

,Houston,LosAngeles,Miami,NewYork,Age,Gender,Subscription_Length_Months,Monthly_Bill,Total_Usage_GB
90679,1.0,0.0,0.0,0.0,43.0,0.0,22.0,75.92,353.0
52212,1.0,0.0,0.0,0.0,45.0,1.0,20.0,57.41,374.0
9203,1.0,0.0,0.0,0.0,66.0,1.0,11.0,99.08,191.0
3942,0.0,1.0,0.0,0.0,44.0,0.0,21.0,34.31,173.0
31003,0.0,0.0,1.0,0.0,52.0,0.0,1.0,91.40,97.0
...,...,...,...,...,...,...,...,...,...
6265,0.0,0.0,1.0,0.0,35.0,0.0,21.0,67.33,235.0
54886,0.0,0.0,0.0,0.0,56.0,0.0,13.0,85.40,347.0
76820,1.0,0.0,0.0,0.0,69.0,0.0,2.0,76.24,321.0
860,0.0,0.0,0.0,0.0,55.0,0.0,12.0,89.19,315.0


In [42]:
# df['Age'] = np.log(df['Age'])
# df['Subscription_Length_Months'] = np.log(df['Subscription_Length_Months'])
df['Monthly_Bill'] = np.log(df['Monthly_Bill'])
df['Total_Usage_GB'] = np.log(df['Total_Usage_GB'])

## model

In [43]:
from keras.layers import BatchNormalization, Dense
from keras.losses import binary_crossentropy
from tensorflow import keras
from keras.callbacks import LearningRateScheduler , EarlyStopping
from keras.activations import relu , sigmoid
from keras import Sequential
from keras.initializers import he_normal

In [44]:
model = Sequential()

model.add(layer=Dense(units=512,activation=relu,kernel_initializer=he_normal))
model.add(layer=Dense(units=332,activation=relu,kernel_initializer=he_normal))
model.add(BatchNormalization())
model.add(Dense(units=128,activation=relu,kernel_initializer=he_normal))
model.add(Dense(units=64,activation=relu,kernel_initializer=he_normal))
model.add(Dense(units=1,activation=sigmoid,name='output_layer'))

In [45]:
def lr_schedule(epoch, lr):
    if epoch < 1:
        return lr
    else:
        return lr * np.exp(-0.1)

# learning rate scheduler callback to descrese the learning rate gradually as the epochs increases So that my alogrithm could not jump out of Global minima.
lr_scheduler = LearningRateScheduler(lr_schedule)


early_stopping = EarlyStopping(
    monitor="accuracy",
    min_delta=0.00001,
    patience=5,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False
)

In [46]:
optimizer = keras.optimizers.Adam(learning_rate=0.0005)

In [47]:
model.compile(optimizer=optimizer, 
               loss=binary_crossentropy, 
                 metrics=['accuracy']) 

In [48]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20,
                    batch_size=64,
                      callbacks=[lr_scheduler,early_stopping]) 

Epoch 1/20


1256/1256 [==============================] - 9s 5ms/step - loss: 0.7005 - accuracy: 0.5001 - val_loss: 0.7269 - val_accuracy: 0.5018 - lr: 5.0000e-04
Epoch 2/20
1256/1256 [==============================] - 7s 6ms/step - loss: 0.6952 - accuracy: 0.5014 - val_loss: 0.6939 - val_accuracy: 0.5006 - lr: 4.5242e-04
Epoch 3/20
1256/1256 [==============================] - 7s 6ms/step - loss: 0.6945 - accuracy: 0.4992 - val_loss: 0.6992 - val_accuracy: 0.5003 - lr: 4.0937e-04
Epoch 4/20
1256/1256 [==============================] - 7s 5ms/step - loss: 0.6938 - accuracy: 0.5042 - val_loss: 0.6933 - val_accuracy: 0.5040 - lr: 3.7041e-04
Epoch 5/20
1256/1256 [==============================] - 7s 5ms/step - loss: 0.6938 - accuracy: 0.5027 - val_loss: 0.6936 - val_accuracy: 0.5017 - lr: 3.3516e-04
Epoch 6/20
1256/1256 [==============================] - 7s 5ms/step - loss: 0.6935 - accuracy: 0.5010 - val_loss: 0.6947 - val_accuracy: 0.4987 - lr: 3.0327e-04
Epoch 7/20
1256/1256 [=======================

## Accuracy not improved after Adding more data aslo